In [ ]:
"""
Análise de sensibilidade (local / OAT) dos parâmetros do modelo ARX (theta)
avaliando o desempenho em malha-fechada com IMPC.

- Perturba cada parâmetro theta_ij em +/- eps (relativo quando possível)
- Simula a malha fechada (IMPC) e computa métricas (SSE, IAE, DU2)
- Calcula sensibilidade normalizada: S = (J+ - J-) / (2*delta) * (theta_ij / J0)

Saídas:
- imprime Top-k parâmetros mais sensíveis
- salva CSV com a tabela completa
"""

import hickle as hkl
import numpy as np
from scipy.optimize import minimize

from lib.read_data import u_desvio as u_true
from lib.read_data import y_desvio as y_true
from lib.read_data import u_max_desvio as u_max
from lib.read_data import u_min_desvio as u_min


# -----------------------------
# Configurações gerais
# -----------------------------
model_file = "../export/ARX.hkl"

ncycles = 80          # use menor p/ rodar sensibilidade mais rápido (ex.: 60-100)
Np = 10
Nc = 3

# pesos do controlador (adequar aos seus)
ny = y_true.shape[1]
nu = u_true.shape[1]

Q = np.full(ny, 1.0)
Q[3] = 30
Q[4] = 30

R = np.array([1.0, 1.0, 0.5, 0.5], dtype=float)  # Δu

ki = np.full(ny, 0.001)  # integrador

# limites de taxa (Δu) por ciclo (já ajuda a evitar dithering)
du_max = np.array([10.0, 10.0, 5.0, 5.0], dtype=float)

# penalização leve em torno de um u nominal (opcional)
use_u_nom_penalty = True
S = np.array([1e-4, 1e-4, 1e-4, 1e-4], dtype=float)

# integrador: anti-windup (clamp)
use_anti_windup = True
ni_clip = 0.5

# setpoints (em DESVIO!)
sp = np.array([2.60473585, 0.1195672, -8.24649468, 8.04911466, 0.16813086], dtype=float)
u_sp = np.array([-70, -45, 5, 25], dtype=float)
u_nom = u_sp.copy()

# sensibilidade
eps = 0.01            # 1% (relativo) quando theta != 0
eps_abs = 1e-4        # passo absoluto quando theta ~ 0
top_k = 15
csv_out = "../export/sensibilidade_theta.csv"


# -----------------------------
# Utilidades do modelo ARX
# -----------------------------
def unpack_theta(theta, ny, nu):
    """
    theta shape: (ny + 2*nu, ny)
    A = theta[:ny,:].T  -> (ny, ny)
    B = theta[ny:,:].T  -> (ny, 2*nu)
    """
    A = theta[:ny, :].T
    B = theta[ny:, :].T
    return A, B


def model_step(A, B, y, u_prev, u_curr):
    u_ext = np.concatenate([u_prev, u_curr])  # (2*nu,)
    return A @ y + B @ u_ext


# -----------------------------
# IMPC (incremental MPC)
# -----------------------------
def _du_to_u_sequence(du_seq, u_old):
    return u_old + np.cumsum(du_seq, axis=0)


def J_impc(du_flat, A, B, y0, sp, u_old, ni, u_nom):
    du_seq = np.asarray(du_flat, dtype=float).reshape((Nc, nu))
    u_seq = _du_to_u_sequence(du_seq, u_old)

    y = np.asarray(y0, dtype=float).copy()
    u_prev = np.asarray(u_old, dtype=float).copy()
    cost = 0.0

    for k in range(Np):
        u_curr = u_seq[k] if k < Nc else u_seq[-1]
        y_next = model_step(A, B, y, u_prev, u_curr)

        e = y_next - sp + ni
        cost += np.sum(Q * (e * e))

        if use_u_nom_penalty:
            cost += np.sum(S * (u_curr - u_nom) ** 2)

        y = y_next
        u_prev = u_curr

    cost += np.sum(R * (du_seq * du_seq))
    return float(cost)


def _constraints_u_bounds(du_flat, u_old):
    du_seq = np.asarray(du_flat, dtype=float).reshape((Nc, nu))
    u_seq = _du_to_u_sequence(du_seq, u_old)

    g = []
    for j in range(Nc):
        u_j = u_seq[j]
        g.append(u_j - u_min)  # >=0
        g.append(u_max - u_j)  # >=0
    return np.concatenate(g)


def IMPC(A, B, y0, sp, u_old, ni, u_nom, du_init=None):
    # chute inicial (warm-start)
    if du_init is None:
        du0 = np.zeros((Nc, nu), dtype=float).flatten()
    else:
        du0 = np.asarray(du_init, dtype=float).reshape((Nc, nu)).flatten()

    cons = [{"type": "ineq", "fun": lambda du_flat, u_old=u_old: _constraints_u_bounds(du_flat, u_old)}]
    bounds = [(-du_max[i % nu], du_max[i % nu]) for i in range(Nc * nu)]

    res = minimize(
        J_impc,
        du0,
        args=(A, B, y0, sp, u_old, ni, u_nom),
        method="SLSQP",
        constraints=cons,
        bounds=bounds,
        options={"maxiter": 250, "ftol": 1e-7, "disp": False},
    )

    du_opt = np.array(res.x, dtype=float).reshape((Nc, nu))
    u_new = u_old + du_opt[0]
    u_new = np.minimum(np.maximum(u_new, u_min), u_max)  # segurança numérica

    return u_new, du_opt


# -----------------------------
# Simulação e métricas
# -----------------------------
def simulate_closed_loop(theta, ncycles=ncycles):
    A, B = unpack_theta(theta, ny, nu)

    y = np.zeros((ncycles, ny), dtype=float)
    u = np.zeros((ncycles, nu), dtype=float)

    y[0] = y_true[0]
    u[0] = u_true[0]

    ni = np.zeros(ny, dtype=float)

    du_prev = None  # warm-start: desloca a sequência Δu no tempo

    for k in range(1, ncycles):
        ni += (y[k - 1] - sp) * ki
        if use_anti_windup:
            ni = np.clip(ni, -ni_clip, ni_clip)

        u_k, du_opt = IMPC(
            A, B,
            y0=y[k - 1],
            sp=sp,
            u_old=u[k - 1],
            ni=ni,
            u_nom=u_nom,
            du_init=du_prev
        )
        u[k] = u_k
        y[k] = model_step(A, B, y[k - 1], u[k - 1], u[k])

        # warm-start: shift (du[1:], du[-1]) para o próximo ciclo
        du_shift = np.vstack([du_opt[1:], du_opt[-1:]])
        du_prev = du_shift

    return y, u



# -----------------------------
# Rodar e salvar
# -----------------------------
theta = hkl.load(model_file)
base_metrics, sens_table = local_sensitivity(theta, metric_name="SSE")

# ordenar por |S_norm| (ou |dJ/dtheta| se S_norm for nan)
def sort_key(row):
    val = row["S_norm"]
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return abs(row["dJ_dtheta"])
    return abs(val)

sens_table_sorted = sorted(sens_table, key=sort_key, reverse=True)

print("\nMétricas base:", base_metrics)
print(f"\nTop {top_k} parâmetros mais sensíveis (por |S_norm|):")
for r in sens_table_sorted[:top_k]:
    print(f"theta[{r['i']},{r['j']}]={r['theta_ij']:.3e}  dJ/dth={r['dJ_dtheta']:.3e}  S_norm={r['S_norm']}")

# salvar CSV (sem pandas)
import csv
with open(csv_out, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=list(sens_table[0].keys()))
    writer.writeheader()
    for row in sens_table_sorted:
        writer.writerow(row)

print(f"\nCSV salvo em: {csv_out}")


[SSE] (0,0) dJ/dth=1.451e+04  S_norm=1.8659684081232022
[SSE] (0,1) dJ/dth=-9.806e+04  S_norm=-0.026784978362596898
[SSE] (0,2) dJ/dth=1.063e+03  S_norm=0.20180115290216474
[SSE] (0,3) dJ/dth=1.923e+01  S_norm=0.010338675136892149
[SSE] (0,4) dJ/dth=2.358e+04  S_norm=0.51918652314804
[SSE] (1,0) dJ/dth=5.853e+02  S_norm=-0.21650263692161567
[SSE] (1,1) dJ/dth=-4.372e+03  S_norm=-0.5891905841800321
[SSE] (1,2) dJ/dth=4.475e+01  S_norm=-0.029339136950515116
[SSE] (1,3) dJ/dth=2.470e+01  S_norm=-0.03428220580942975
[SSE] (1,4) dJ/dth=1.172e+03  S_norm=-0.02326591027613741
[SSE] (2,0) dJ/dth=-4.025e+04  S_norm=0.16303481776696946
[SSE] (2,1) dJ/dth=3.101e+05  S_norm=-0.6448748907533366
[SSE] (2,2) dJ/dth=-3.550e+03  S_norm=-0.19037498898475816
[SSE] (2,3) dJ/dth=4.067e+02  S_norm=-0.0015770139893930687
[SSE] (2,4) dJ/dth=-7.512e+04  S_norm=0.060566034150971206
[SSE] (3,0) dJ/dth=4.529e+04  S_norm=0.21178537364104516
[SSE] (3,1) dJ/dth=-3.430e+05  S_norm=-0.19326592853288418
[SSE] (3,2) dJ/